# Feature Engineering für eine Drohnen Fernsteuerung, die mit einem MPU6050 ausgestattet ist

## 1. Einlesen der Datei und überprüfen, ob die Datenreihen vollständig sind

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix as scatmat
import os


# Absolute Pfad zur Datei extrahieren
#ziel_pfad = 'D:\measurements'
ziel_pfad = '../Daten/'

# CSV-Datei einlesen
data = pd.read_csv(os.path.join(ziel_pfad, 'mpu6050_lennard_run1.csv'), sep=',', decimal='.')
data2 = pd.read_csv(os.path.join(ziel_pfad, 'mpu6050_lennard_run2.csv'), sep=',', decimal='.')
data = pd.concat([data, data2], ignore_index=True)

# Die ersten Zeilen anzeigen
data.head()

# Informationen über den Datensatz anzeigen
data.info()

# Sind alle Klassen gleich stark vertreten?
data['RuheState'].value_counts()

# Gibt es Auffälligkeiten bei der Verteilung der Werte?

data.describe()

## 2. Irrelevante Merkmale entfernen

In [ ]:
# data = data.drop(['user', 'gender'], axis=1)

# 3. Visualisierung der Verteilung der numerischen Werte

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
data.hist(bins=50, figsize=(20, 15))
plt.show()

## 4. Überprüfen, ob es fehlerhafte Werte gibt (Bspw.: in Spalte 'AccX')

In [ ]:
pd.set_option('display.max_rows', 300)
data['AccX'].value_counts()

## 5. Fehlerhaften Wert korrigieren (Beispiel)

In [ ]:
#data.loc[data['z4'] == '-14420-11-2011 04:50:23.713', 'z4'] = -144

# Spalte 'z4' in numerischen Typ umwandeln
#data['z4'] = pd.to_numeric(data['z4'])

# Informationen über den Datensatz erneut anzeigen
data.info()

## MinMaxScaler 

In [ ]:
# Skalieren

from sklearn.preprocessing import MinMaxScaler
#data.drop('class', axis=1, inplace=True) # Ist bereits in Schritt 11 erfolgt
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
#print("data_scaled.shape:", data_scaled.shape, "\ndata_scaled:\n", data_scaled) # uncomment to visualize details

# Robust Scaler 

In [ ]:
# Das ist eine Alternative zum MinMaxScaler, der robust gegenüber auffällig hohen / tiefen Werten ist

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
data_scaled = scaler.fit_transform(data)

# Standardabweichung berechnen

In [ ]:
# Blockgröße für Berechnungen
chunk_size = 30

data_scaled_pp_df = pd.DataFrame(data_scaled, columns=data.columns)

# Anzahl der Blöcke berechnen
num_chunks = len(data_scaled_pp_df) // chunk_size

# Verarbeitung in Blöcken durchführen
processed_data_list = []

for i in range(num_chunks):
    start_index = i * chunk_size
    end_index = (i + 1) * chunk_size

    chunk = data_scaled_pp_df.iloc[start_index:end_index, :]

    # Standardabweichung für ausgewählte Spalten berechnen
    selected_columns = ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'AngleX', 'AngleY', 'AngleZ', 'AccAngleX', 'AccAngleY']
    std_deviation = chunk[selected_columns].std().to_frame().transpose()

    # Mittelwert für andere Spalten berechnen
    other_columns = ['RuheState', 'FernstState', 'TranspState']
    mean_values = chunk[other_columns].mean().to_frame().transpose()

    # Neue Daten erstellen
    processed_data = pd.concat([std_deviation, mean_values], axis=1)
    
    processed_data_list.append(processed_data)

# Alle berechneten Blöcke zusammenführen
result_data = pd.concat(processed_data_list, ignore_index=True)

result_data

## 6. Aufteilung des Datensatzes in Trainings- und Validierungsdaten

In [ ]:
# Aufteilung in Trainings- und Validierungsdatensätze
train, val = train_test_split(result_data, test_size=0.1, random_state=42)

# Anzahl der Datensätze anzeigen
len(train), len(val)

# Kopie des Trainingsdatensatzes für Modifikationen
data_scaled = train.copy()


## 7. Begrenzung der Auswahl auf die Sensorspalten

In [ ]:
data.hist(bins=50, figsize=(20, 15))
plt.show()

## 8. Prüfung der Korrelationen für verschiedene Sensoren

In [ ]:
sensors = ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'AngleX', 'AngleY', 'AngleZ', 'RuheState', 'FernstState', 'TranspState']

for sensor_group in [sensors[:3], sensors[3:6], sensors[6:9]]:
    scatmat(data[sensor_group], figsize=(15, 10))
    plt.show()

## 9. Visualisierung der Klassenzuordnung im Scatterplot

In [ ]:
# Gruppiere nach den Klassenattributen und zähle die Anzahl der Vorkommen
class_counts = data.groupby(['RuheState', 'FernstState', 'TranspState']).size().reset_index(name='count')

# Anzeige aller Kombinationen
for i, row in class_counts.iterrows():
    print(f"Klasse {i}: Ruhe={row['RuheState']}, Fernsteuerung={row['FernstState']}, Transport={row['TranspState']} - Anzahl: {row['count']}")


## 10. Scatterplots der unskalierten Daten für verschiedene Sensorpaare

In [ ]:
# Erstelle eine temporäre Spalte 'class' basierend auf One-Hot-Encoding
data['class'] = data.apply(lambda row: 1 if row['RuheState'] == 1 else (2 if row['FernstState'] == 1 else 3), axis=1)

# Visualisiere Scatterplots für jedes Sensor-Paar
sensor_pairs = [('AccX', 'AccY'), ('GyroX', 'GyroY'), ('GyroZ', 'AngleX'), ('AngleY', 'AngleZ')]
for pair in sensor_pairs:
    data.plot(kind="scatter", x=pair[0], y=pair[1], alpha=0.1, c='class', cmap=plt.get_cmap('jet'))
    #plt.title(f'Unskalierte Werte - {pair[0]} vs. {pair[1]}')
    plt.show()

# Entferne die temporäre 'class'-Spalte
data.drop('class', axis=1, inplace=True)


## 11. One Hot encoding der Klasse durchführen, wenn noch nicht geschehen (Beispiel)

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#encoder = OneHotEncoder(categories='auto');
#data_label_matrix = data["class"].values.reshape(-1, 1)
#data_labels_1hot = encoder.fit_transform(data_label_matrix)
#data_labels_1hot.shape

## 14. Skatterplots von Messwertpaaren mit vorheriger Skalierung

In [ ]:
# Skalierte Werte in einem neuen DataFrame mit den ursprünglichen Spaltennamen
data_scaled_df = pd.DataFrame(data_scaled, columns=data.columns)


# Erstelle eine temporäre Spalte 'class' basierend auf One-Hot-Encoding
data_scaled_df['class'] = data_scaled_df.apply(lambda row: 1 if row['RuheState'] == 1 else (2 if row['FernstState'] == 1 else 3), axis=1)


# Visualisiere Scatterplots für jedes Sensor-Paar mit den skalierten Werten und den originalen Klassen
sensor_pairs = [('AccX', 'AccY'), ('GyroX', 'GyroY'), ('GyroZ', 'AngleX'), ('AngleY', 'AngleZ')]
for pair in sensor_pairs:
    data_scaled_df.plot(kind="scatter", x=pair[0], y=pair[1], alpha=0.1, c=data_scaled_df['class'], cmap=plt.get_cmap('jet'))
    plt.title(f'Skalierte Werte - {pair[0]} vs. {pair[1]}')
    plt.show()

# Entferne die temporäre 'class'-Spalte
#data_scaled_df.drop('class', axis=1, inplace=True)


In [ ]:
from itertools import combinations

# Annahme: 'data' ist dein ursprüngliches DataFrame mit den Daten

# Skalierte Werte in einem neuen DataFrame mit den ursprünglichen Spaltennamen
data_scaled_df = pd.DataFrame(data_scaled, columns=data.columns)

# Erstelle eine temporäre Spalte 'class' basierend auf One-Hot-Encoding
data_scaled_df['class'] = data_scaled_df.apply(lambda row: 1 if row['RuheState'] == 1 else (2 if row['FernstState'] == 1 else 3), axis=1)

# Liste aller Labels
all_labels = ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'AngleX', 'AngleY', 'AngleZ', 'RuheState', 'FernstState', 'TranspState']

# Erstelle alle möglichen Kombinationen von Label-Paaren
label_pairs = list(combinations(all_labels[0:9], 2))
print(len(label_pairs))

# Anzahl der Zeilen und Spalten für die Subplots
num_rows = 9  # Du kannst dies anpassen, um die Anzahl der gewünschten Zeilen zu ändern
num_cols = 4  # Du kannst dies anpassen, um die Anzahl der gewünschten Spalten zu ändern

# Erstelle Subplots
#plt.figure(figsize=(10, 8))
fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 40))

# Iteriere durch Label-Paare und erstelle Scatterplots
for i in range(num_rows):
    for j in range(num_cols):
        label_pair = label_pairs[i * num_cols + j]
        axs[i, j].scatter(data_scaled_df[label_pair[0]], data_scaled_df[label_pair[1]], alpha=0.1, c=data_scaled_df['class'], cmap=plt.get_cmap('jet'))
        axs[i, j].set_title(f'Skalierte Werte - {label_pair[0]} vs. {label_pair[1]}')

# Verbessere das Layout
plt.tight_layout()
plt.show()


# 15. Vorverarbeitung

In [ ]:
#

# 16. Training des kNN

In [ ]:
from sklearn import neighbors
import numpy as np

num_features = 14
X = np.zeros([len(data_scaled_df), num_features])
y = data_scaled_df['class']

k = 5
clf = neighbors.KNeighborsClassifier(n_neighbors=k)
features = data_scaled_df.drop(columns='RuheState').drop(columns='class').drop(columns='TranspState').drop(columns='FernstState')
clf.fit(features, y)

# 17. Prediction

In [ ]:
# Prediction mit Test-Array
y_predict = clf.predict(features)

acc = np.mean(y_predict == (data_scaled_df['class']).astype(int).values)
acc

In [ ]:
# Prediction mit Validation-Array

# Skalierte Werte in einem neuen DataFrame mit den ursprünglichen Spaltennamen
val_df = pd.DataFrame(val, columns=data.columns)

# Erstelle eine temporäre Spalte 'class' basierend auf One-Hot-Encoding
val_df['class'] = val_df.apply(lambda row: 1 if row['RuheState'] == 1 else (2 if row['FernstState'] == 1 else 3), axis=1)

val_labels = val_df['class']
val_data = val_df.drop(columns='RuheState').drop(columns='class').drop(columns='TranspState').drop(columns='FernstState')

# Prediction
prediction = clf.predict(val_data)

acc = np.mean(prediction == val_labels.astype(int).values)
acc

# Python MQTT Client

In [ ]:
live_data = {'AccX' :[],
        'AccY' : [],
        'AccZ' : [],
        'GyroX' : [],
        'GyroY' : [],
        'GyroZ' : [],
        'AngleX' : [],
        'AngleY' : [],
        'AngleZ' : [],
        'AccAngleX' : [],
        'AccAngleY' : [],     
        'RuheState' : 0,
        'FernstState' : 0,
        'TranspState' : 0,}

live_data_df = pd.DataFrame(live_data)


live_data_df.describe()

# MQTT Live-Prediction

In [ ]:
import paho.mqtt.client as mqtt
from IPython.display import clear_output


live_data_df = pd.DataFrame(live_data)

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    # client.subscribe("$SYS/#")

# The callback for when a PUBLISH message is received from the server.
counter = 0 # counts how many times angleZ is received 

def on_message(client, userdata, msg):
    # print(msg.topic+" "+str(msg.payload))
    global accX, accY, accZ, gyroX, gyroY, gyroZ, angleX, angleY, angleZ, accAngleX, accAngleY, live_data_df
    global counter
    if(msg.topic == 'accelerationX'):
        accX = msg.payload.decode('UTF-8')
        #print(accX)
    elif(msg.topic == 'accelerationY'):
        accY = msg.payload.decode('UTF-8')
        #print(accY)
    elif(msg.topic == 'accelerationZ'):
        accZ = msg.payload.decode('UTF-8')
    
    elif(msg.topic == 'gyrometerX'):
        gyroX = msg.payload.decode('UTF-8')
    elif(msg.topic == 'gyrometerY'):
        gyroY = msg.payload.decode('UTF-8')
    elif(msg.topic == 'gyrometerZ'):
        gyroZ = msg.payload.decode('UTF-8')

    elif(msg.topic == 'angleX'):
        angleX = msg.payload.decode('UTF-8')
    elif(msg.topic == 'angleY'):
        angleY = msg.payload.decode('UTF-8')
    elif(msg.topic == 'angleZ'):
        angleZ = msg.payload.decode('UTF-8')
    elif(msg.topic == 'accAngleX'):
        accAngleX = msg.payload.decode('UTF-8')
    elif(msg.topic == 'accAngleY'):
        accAngleY = msg.payload.decode('UTF-8')
        counter += 1

        new_row = {
        'AccX' :[accX],
        'AccY' : [accY],
        'AccZ' : [accZ],
        'GyroX' : [gyroX],
        'GyroY' : [gyroY],
        'GyroZ' : [gyroZ],
        'AngleX' : [angleX],
        'AngleY' : [angleY],
        'AngleZ' : [angleZ],
        'AccAngleX' : [accAngleX],
        'AccAngleY' : [accAngleY],
        'RuheState' : 0,
        'FernstState' : 0,
        'TranspState' : 0,
        }
        
        # convert new Data to DataFrame
        new_row = pd.DataFrame(new_row)

        # append new row to live_data_df
        live_data_df = pd.concat([new_row, live_data_df], ignore_index=True)

        # after 30 measurements, call the classifier
        if counter == 10:
            counter = 0
            live_data_scaled = pd.DataFrame(scaler.transform(live_data_df), columns=data.columns)   # scaling our data
            live_data_features = live_data_scaled.drop(columns='RuheState').drop(columns='TranspState').drop(columns='FernstState')
            # live_data_features = live_data_scaled[['AccY', 'GyroX']]

            std_df = live_data_features[selected_columns].std().to_frame().transpose()
            #print(std_df)

            #std_df = pd.DataFrame([['AccY', 'GyroX'], [std_accy, std_gyrox]])

            live_predict = clf.predict(std_df)
            print('Prediction: ', str(live_predict.mean()))
            if round(live_predict.mean()) == 1:
                print('Ruhe') 
            elif round(live_predict.mean()) == 2:
                print('Fernsteuerung') 
            elif round(live_predict.mean()) == 3:
                print('Transport')


            # Plotting 
            plt.figure(figsize=(10, 7))
            plt.scatter(train['AccY'], train['GyroX'],marker=".", alpha=0.3, c=data_scaled_df['class'], cmap=plt.get_cmap('rainbow'))
            plt.scatter(std_df['AccY'], std_df['GyroX'], marker="P", alpha=0.9, c=4, cmap=plt.get_cmap('jet'))
            #print(live_data_features)
            # axes = plt.gca()
            # axes.set_xlim([-5, 5])
            # axes.set_ylim([-20, 20])

            ## ACHTUNG: Glaube die Labels der Prediction passen nicht zu den Features. Im Plot sind falsche Farben in falschen Bereichen zu erkennen

            # Verbessere das Layout
            plt.tight_layout()
            plt.show()

            clear_output(wait=True)

            # Reset live_predict and live_data_df
            live_predict = None
            live_data_df = live_data_df.head(20) # keep 20 rows of the currenct frame

    

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("192.168.137.1", 1883, 60)
client.subscribe("ESP_message", 0)
client.subscribe("accelerationX", 0)
client.subscribe("accelerationY", 0)
client.subscribe("accelerationZ", 0)
client.subscribe("gyrometerX", 0)
client.subscribe("gyrometerY", 0)
client.subscribe("gyrometerZ", 0)
client.subscribe("angleX", 0)
client.subscribe("angleY", 0)
client.subscribe("angleZ", 0)
client.subscribe("accAngleX", 0)
client.subscribe("accAngleY", 0)


# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

In [ ]:
# Standardabweichung

data_features = data[['AccY', 'GyroX']]

std_accy = data_features[['AccY']].std()
std_gyrox = data_features[['GyroX']].std()


In [ ]:
x = np.linspace(0, 1, 100)
X0, X1 = np.meshgrid(x, x)
Xstack = np.dstack((X0, X1))
Y = np.zeros(X0.shape)
for r in range(100):
    for c in range(100):
        Y[r, c] = clf.predict(Xstack[r, c, :].reshape(1, -1))

plt.contourf(X0, X1, Y, cmap=plt.get_cmap('jet'))


# Ansatz Classifier nur mit Acc Y und Gyro X

In [ ]:
# Training

num_features = 14
labels = data_scaled_df['class']

k = 5
clf = neighbors.KNeighborsClassifier(n_neighbors=k)
features = data_scaled_df[['AccY', 'GyroX']]
clf.fit(features, labels)

In [ ]:
# Prediction

val_labels = val_df['class']
val_features = val_df[['AccY', 'GyroX']]

# Prediction
prediction = clf.predict(val_features)

acc = np.mean(prediction == val_labels.astype(int).values)
acc

# Confusion Matrix

In [ ]:
import seaborn as sns
import pandas as pd

def confusion_matrix(results, labels):
    data = {'y_Actual': results.astype(np.uint8), 'y_Predicted': labels}
    df = pd.DataFrame(data, columns=['y_Actual', 'y_Predicted'])
    cm = pd.crosstab(df['y_Actual'], df['y_Predicted'])
    cm = cm.astype(np.float32) / cm.sum(axis=1).values[:, np.newaxis] * 100.0
    sns.heatmap(cm, annot=True, fmt='.1f', annot_kws={"size": 8})
    plt.title('Normalized Confusion Matrix in %')
    plt.show()

confusion_matrix(prediction, val_labels.astype(int).values) # val confusion Matrix
confusion_matrix(y_predict, (data_scaled_df['class']).astype(int).values)   # train confusion Matrix

# Confusion Matrix mit verarbeiteten Daten

In [ ]:
# Confusion Matrix mit verarbeiteten Daten

confusion_matrix(prediction, val_labels.astype(int).values) 
confusion_matrix(y_predict, (data_scaled_df['class']).astype(int).values)   # train confusion Matrix